In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip -q -o '/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip'

In [ ]:
!unzip -q -o '/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip'

In [ ]:
# test/10007.jpg

In [ ]:
# train/cat.4511.jpg

In [ ]:
from PIL import Image

Image.open('train/cat.4511.jpg')

In [ ]:
import glob

train = pd.DataFrame({'path' : glob.glob('train/*')})
train.head(2)

In [ ]:
a = train['path'].apply(lambda x: x.split('/')[1])
a

In [ ]:
train['target'] = train['path'].apply(lambda x: x.split('/')[1].split('.')[0])

In [ ]:
train

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

idg2 = ImageDataGenerator(horizontal_flip = True,
                        brightness_range = [0.2, 1.0])

idg = ImageDataGenerator()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_valid = train_test_split(train, test_size = 0.2,
                                   random_state = 42, stratify = train['target'])

In [ ]:
x_valid['target'].value_counts()

In [ ]:
train['target'].value_counts()

In [ ]:
train_generator = idg2.flow_from_dataframe(x_train, x_col = 'path',
                                         y_col = 'target', target_size = (100,100))

In [ ]:
valid_generator = idg.flow_from_dataframe(x_valid, x_col = 'path',
                                         y_col = 'target', target_size = (100,100))

In [ ]:
train_generator

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize = (12,12))
for i in range(0,15):
    plt.subplot(5,3,i+1)
    for x, y in train_generator:
        image = x[0]
        plt.imshow(image.astype('uint8'))
        break
# plt.tight_layout()
plt.show()

In [ ]:
from tensorflow.keras import Sequential

from tensorflow.keras.layers import *

from tensorflow.keras.applications import EfficientNetB1

In [ ]:
eb0 = EfficientNetB1(include_top = False, pooling = 'avg')

In [ ]:
model = Sequential()

model.add(eb0)
model.add(Dense(2, activation = 'softmax'))

from tensorflow.keras.optimizers import SGD

model.compile(metrics = ['acc'], loss = 'categorical_crossentropy',
             optimizer = SGD(momentum = 0.9, nesterov = True, 
                            lr = 0.01))

In [ ]:
model.summary()

In [ ]:
model.fit(train_generator, epochs = 5,
         validation_data = valid_generator)

In [ ]:
test = pd.DataFrame({'path' : glob.glob('test/*')})

test

In [ ]:
test_generator = idg.flow_from_dataframe(test, x_col = 'path',
                                        y_col = None, 
                                        class_mode = None,
                                        target_size = (100,100),
                                        shuffle = False)

In [ ]:
result = model.predict(test_generator, verbose = 1)

In [ ]:
result

In [ ]:
from keras.preprocessing.image import load_img


n = 10
for i, (index, row) in enumerate(test.iterrows()):
    if i >= n:
        break
    fig = plt.figure(figsize=(8, 32))
    img = load_img(row['path'], target_size=(100, 100))
    subfig = fig.add_subplot(n, 1, i + 1)
    pred = result[i][0]
    pred_label = 'cat' if pred > 0.5 else 'dog'
    pred = pred if pred > 0.5 else 1-pred
    plt.title('Looks like a {0} with probability {1}'.format(pred_label, pred))
    f = plt.imshow(img)
    f.axes.get_xaxis().set_visible(False)
    f.axes.get_yaxis().set_visible(False)

In [ ]:
sub = pd.read_csv('/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')
sub

In [ ]:
sub['id'] = test['path'].apply(lambda x : x.split('/')[1].split('.')[0] )

In [ ]:
sub['label'] = result[:,1].clip(0.005, 0.995)
sub

In [ ]:
sub.to_csv('aug-sgd.csv', index = 0)